In [677]:
import pandas as pd
import numpy as np
import os
import re
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

from linearmodels.system import SUR
from pathlib import Path
from IPython.display import display
from scipy.stats import pearsonr
from collections import OrderedDict
from sklearn.linear_model import LinearRegression
from linearmodels.system.results import WaldTestStatistic


import warnings
warnings.filterwarnings('ignore')

In [636]:
root = Path(os.getcwd())

In [637]:
def set_timeframe(data, start_date, end_date):
    data = data.loc[data.index.get_level_values('date') >= start_date]
    data = data.loc[data.index.get_level_values('date') <= end_date]
    return data

In [638]:
predictors = pd.read_parquet(root / 'data' / 'predictors.parquet.brotli')

In [639]:
predictors = set_timeframe(predictors, '1925-01-01', '2022-01-01')

# Handling Data

In [640]:
regression = predictors.copy(deep=True)

In [641]:
regression = regression.dropna()

In [642]:
# Bubble dates from paper
bubble_date = pd.read_csv(root / 'data' / 'bubbles.csv')
bubble_date = bubble_date.applymap(lambda s: s.lower() if type(s) == str else s)
bubble_date.date = pd.to_datetime(bubble_date.date)
bubble_date.date = bubble_date.date.dt.to_period('M').dt.to_timestamp('M')
bubble_date = bubble_date.set_index(['date', 'ind'])

In [643]:
# Add excess returns
regression['rf_two_vwr_ind'] = regression['two_vwr_ind'] - regression['rf']/100
regression['market_two_vwr_ind'] = regression['two_vwr_ind'] - regression['market_return']

In [644]:
regression.index.get_level_values(1).unique()

Index(['autos', 'beer', 'boxes', 'chems', 'chips', 'coal', 'drugs', 'elceq',
       'fin', 'food', 'fun', 'hshld', 'mach', 'mines', 'oil', 'rtail', 'ships',
       'smoke', 'steel', 'telcm', 'trans', 'util', 'bldmt', 'labeq', 'rlest',
       'txtls', 'aero', 'hardw', 'bussv', 'clths', 'cnstr', 'medeq', 'agric',
       'meals', 'insur', 'banks', 'books', 'whlsl', 'toys', 'persv', 'other',
       'paper', 'rubbr', 'gold', 'guns', 'hlth', 'soda', 'softw', 'fabpr'],
      dtype='object', name='ind')

# Creating Table 6 (Bubbles for Fama)

In [645]:
def get_post_bubble_data(df, date, ind, months=24):
    start_date = pd.to_datetime(date)
    end_date = start_date + pd.DateOffset(months=months)
    mask = (df.index.get_level_values('date') > start_date) & (df.index.get_level_values('date') <= end_date) & (df.index.get_level_values('ind') == ind)
    return df[mask]

In [646]:
def set_correct_format(results):
    # 2. Extract series data into separate columns for 'b'
    df = pd.DataFrame(results)
    # Extract the coefficients ('b') and t-values ('t')
    df_b = df['b'].apply(pd.Series)
    df_t = df['t'].apply(pd.Series)
    # Rename columns to indicate they contain coefficients or t-values
    df_b.columns = [f"b_{col}" for col in df_b.columns]
    df_t.columns = [f"t_{col}" for col in df_t.columns]
    df_final = pd.concat([df.drop(['b', 't'], axis=1), df_b, df_t], axis=1)
    # Melt the dataframe for betas
    df_melted_b = df_final.melt(id_vars=['ind', 'predictor', 'r'], 
                          value_vars=[col for col in df_final.columns if col.startswith('b_')],
                          var_name='b_type', value_name='b')
    df_melted_b_filtered = df_melted_b[df_melted_b['b_type'] == 'b_' + df_melted_b['predictor']]
    df_melted_b_filtered = df_melted_b_filtered[['ind', 'predictor', 'r', 'b']]
    # Melt the dataframe for t-values
    df_melted_t = df_final.melt(id_vars=['ind', 'predictor', 'r'], 
                                value_vars=[col for col in df_final.columns if col.startswith('t_')],
                                var_name='t_type', value_name='t')

    df_melted_t_filtered = df_melted_t[df_melted_t['t_type'] == 't_' + df_melted_t['predictor']]
    df_melted_t_filtered = df_melted_t_filtered[['ind', 'predictor', 't']]
    # Merge the two melted dataframes on 'ind' and 'predictor'
    df_combined = pd.merge(df_melted_b_filtered, df_melted_t_filtered, on=['ind', 'predictor'])
    df_combined['f'] = df['f'].values
    df_combined['p'] = df['p'].values
    df_combined = df_combined.set_index(['ind', 'predictor'])
    return df_combined

In [647]:
def run_regression(data, X, y, cluster):
    total = []
    for ind, df in data.groupby('ind'):
        results = []
        for predictor in X:
            # OLS
            X_val = df[[predictor]]
            y_val = df[[y]]
            X_val = sm.add_constant(X_val)

            # Fit model with robust standard errors clustered by calendar year
            model_year_clustered = sm.OLS(y_val, X_val).fit(cov_type='cluster', cov_kwds={'groups': df[cluster]})

            # Metrics
            results.append({
                'ind': ind,
                'predictor': predictor,
                'b': model_year_clustered.params,
                't': model_year_clustered.tvalues,
                'r': model_year_clustered.rsquared,
                'f': model_year_clustered.fvalue,
                'p': model_year_clustered.f_pvalue
            })
        total.append(set_correct_format(results))
    total = pd.concat(total, axis=0)
    return total

### Regression

In [648]:
two_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in bubble_date.index.tolist()])
two_regression['Year'] = two_regression.index.get_level_values('date').year

In [649]:
characteristics = ['vol_vw', 'vol_vw_change', 'turnover_vw', 'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 'revenue_market_rank', 'cape', 'acc']

In [650]:
raw_regress = run_regression(two_regression, characteristics, 'two_vwr_ind', 'Year')
market_regress = run_regression(two_regression, characteristics, 'market_two_vwr_ind', 'Year')
rf_regress = run_regression(two_regression, characteristics, 'rf_two_vwr_ind', 'Year')

### Table

In [755]:
def collect_stats(formater, data, col_name):
    table_format = formater
    collect_r = []
    collect_b = [] 
    collect_t = []
    collect_f = []
    collect_p = []
    for _, df in data.groupby('predictor'):
        collect_r.append(df.r.mean())
        collect_b.append(df.b.mean())
        collect_t.append(df.t.mean())
        collect_f.append(df.f.mean())
        collect_p.append(df.p.mean())
    table_format[f'{col_name} b'] = collect_b
    table_format[f'{col_name} [t]'] = collect_t
    table_format[f'{col_name} R-squre'] = collect_r
    return table_format, np.array(collect_f).mean(), np.array(collect_p).mean()

In [761]:
format_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_rf_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_market_table6 = pd.DataFrame(index=['Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])

In [762]:
raw = collect_stats(format_table6, raw_regress, '24mo Raw Return')
market = collect_stats(format_market_table6, market_regress, '24mo Net of Market Return')
rf = collect_stats(format_rf_table6, rf_regress, '24mo Net of Risk-Free Return')

In [763]:
table6 = pd.concat([raw[0], rf[0], market[0]], axis=1)

In [767]:
table6.loc['Joint F-stat'] = [np.nan, raw[1], np.nan, np.nan, rf[1], np.nan, np.nan, market[1], np.nan]

In [768]:
table6.loc['p-value (Prob>F)'] = [np.nan, raw[2], np.nan, np.nan, rf[2], np.nan, np.nan, market[2], np.nan]

In [769]:
table6

,24mo Raw Return b,24mo Raw Return [t],24mo Raw Return R-squre,24mo Net of Risk-Free Return b,24mo Net of Risk-Free Return [t],24mo Net of Risk-Free Return R-squre,24mo Net of Market Return b,24mo Net of Market Return [t],24mo Net of Market Return R-squre
Volatility (VW),-0.119365,0.885975,0.213610,-0.119496,1.520143,0.215467,-0.124425,-0.032211,0.192817
Volatility (VW)- 1yr-Δ,-0.046160,0.202197,0.129266,0.022932,0.200671,0.126220,-0.023278,0.190754,0.125033
Turnover (VW),-0.306698,-0.269031,0.040023,-0.327395,-0.396283,0.040048,0.155741,1.537869,0.084189
Turnover (VW)- 1yr-Δ,-0.973080,-0.034205,0.165947,-0.715200,0.070248,0.163985,-1.357754,0.009834,0.161538
Firm Age (VW),-0.030103,-3.519238,0.207721,-0.030098,-2.061497,0.210509,-0.031236,-0.406322,0.213608
Age tilt,1.148636,0.234167,0.077045,0.942605,0.168645,0.076562,1.560263,0.427479,0.081633
% Issuers,-28.254872,2.208625,0.179936,-23.626813,2.017198,0.178457,-58.357740,0.593119,0.178802
Book to Market (VW),5.061009,1.822173,0.122375,5.146341,1.828863,0.121615,4.556176,1.107208,0.118698
Sales Growth,-0.035812,-0.007139,0.090569,-0.033765,-0.016808,0.088590,-0.031550,0.022659,0.089829
CAPE,-0.837421,0.213776,0.066391,-1.015563,0.171153,0.065083,0.218327,0.735241,0.070930


# Creating Table 4 (Bubbles for Fama)

In [656]:
all_bubble = regression[regression.index.isin(bubble_date.index)]

In [657]:
table4 = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                           'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                           'CAPE', 'Acceleration'])

In [658]:
column_order = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 
                'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 
                'revenue_market_rank', 'cape', 'acc']

In [659]:
all_ind_mean = regression[column_order].mean().values
all_ind_std = regression[column_order].std().values
table4['All industry-years Mean'] = all_ind_mean
table4['All industry-years STD'] = all_ind_std

In [660]:
run_up_mean = np.array(all_bubble[column_order].mean())
run_up_std = np.array(all_bubble[column_order].std())
table4['Run-ups Mean'] = run_up_mean
table4['Run-ups STD'] = run_up_std

### Detecting crashes

In [661]:
# Function to check if a crash occurred
def did_crash_occur(data, run_up_date):
    two_year_end = run_up_date + pd.DateOffset(years=2)
    post_runup_data = data.loc[run_up_date:two_year_end]
    
    # Calculate cumulative return from run-up date
    post_runup_data['cumulative_return'] = (1 + post_runup_data['vwr_ind']).cumprod() - 1

    # Check if there was any point where the drawdown from the max was 40% or more
    if any(post_runup_data['cumulative_return'] <= (post_runup_data['cumulative_return'].cummax() - 0.4)):
        return 1  # Crash occurred
    return 0  # No crash

In [662]:
run_up_check_crash = pd.DataFrame(columns=['date', 'ind', 'crash'])

for (run_up_date, industry) in bubble_date.index:
    crash = did_crash_occur(regression.xs(industry, level=1), run_up_date)
    run_up_check_crash = pd.concat([run_up_check_crash, pd.DataFrame({'date': [run_up_date], 'ind': [industry], 'crash': [crash]})], ignore_index=True)
    
run_up_check_crash = run_up_check_crash.set_index(['date', 'ind'])
no_crash = run_up_check_crash.loc[run_up_check_crash.crash==0]
crash = run_up_check_crash.loc[run_up_check_crash.crash==1]

In [663]:
crash_bubble = regression[regression.index.isin(crash.index)]
run_up_crash_mean = np.array(crash_bubble[column_order].mean())
run_up_crash_std = np.array(crash_bubble[column_order].std())
table4['Run-ups with Crash Mean'] = run_up_crash_mean
table4['Run-ups with Crash STD'] = run_up_crash_std

In [664]:
no_crash_bubble = regression[regression.index.isin(no_crash.index)]
run_up_no_crash_mean = np.array(no_crash_bubble[column_order].mean())
run_up_no_crash_std = np.array(no_crash_bubble[column_order].std())
table4['Run-ups with no Crash Mean'] = run_up_no_crash_mean
table4['Run-ups with no Crash STD'] = run_up_no_crash_std

### Regression

In [665]:
table4['Crash minus no Crash Mean'] = table4['Run-ups with Crash Mean'] - table4['Run-ups with no Crash Mean']

In [666]:
characteristics = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 'turnover_vw_change', 'age_market_rank', 'age_tilt', 'issuance', 'btm_vwr', 'revenue_market_rank', 'cape', 'acc']

In [667]:
no_crash_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in no_crash.index.tolist()])
no_crash_regression['Year'] = no_crash_regression.index.get_level_values('date').year
no_crash_regression['Ind'] = no_crash_regression.index.get_level_values('ind')
no_crash_regression['Year_Ind'] = no_crash_regression['Year'].astype(str) + '_' + no_crash_regression['Ind'].astype(str)

In [668]:
crash_regression = pd.concat([get_post_bubble_data(regression, date, ind) for date, ind in crash.index.tolist()])
crash_regression['Year'] = crash_regression.index.get_level_values('date').year
crash_regression['Ind'] = crash_regression.index.get_level_values('ind')
crash_regression['Year_Ind'] = crash_regression['Year'].astype(str) + '_' + crash_regression['Ind'].astype(str)

In [669]:
no_crash_stats = run_regression(no_crash_regression, characteristics, 'vwr_ind', 'Year_Ind')
crash_stats = run_regression(crash_regression, characteristics, 'vwr_ind', 'Year_Ind')

In [670]:
format_table4_no_crash = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])
format_table4_crash = pd.DataFrame(index=['Past 2-year Return', 'Excess Past 2-year Return', 'Volatility (VW)', 'Volatility (VW)- 1yr-Δ', 'Turnover (VW)',
                             'Turnover (VW)- 1yr-Δ', 'Firm Age (VW)', 'Age tilt', ' % Issuers', 'Book to Market (VW)', 'Sales Growth',
                             'CAPE', 'Acceleration'])

In [671]:
no_crash_output = collect_stats(format_table4_no_crash, no_crash_stats, 'No Crash')
crash_output = collect_stats(format_table4_crash, crash_stats, 'Crash')

In [672]:
table4['Crash minus no Crash [t]'] = crash_output['Crash [t]'] - no_crash_output['No Crash [t]']

### SUR

In [733]:
def get_stats(data):
    sur_columns = ['two_vwr_ind', 'rf_two_vwr_ind', 'vol_vw', 'vol_vw_change', 'turnover_vw', 
                   'turnover_vw_change', 'age_market_rank', 'issuance', 'age_tilt', 'btm_vwr', 
                   'revenue_market_rank', 'cape', 'acc']
    f = []
    p = []
    for col in sur_columns:
        y = df['vwr_ind']
        X = sm.add_constant(df[col].astype(float))
        model = sm.OLS(y, X).fit()
        f.append(model.fvalue)
        p.append(model.f_pvalue)
    return np.array(f).mean(), np.array(p).mean()
    
def run_sur_all(data):
    f_total = []
    p_total = []
    for ind, df in data.groupby('ind'):
        test = get_stats(df)
        f_total.append(test[0])
        p_total.append(test[1])
    return np.array(f_total).mean(), np.array(p_total).mean()

In [740]:
# crash_sur_stats = run_sur_all(crash_regression)
# no_crash_sur_stats = run_sur_all(crash_regression)
joint_stats = run_sur_all(regression)
f = joint_stats[0]
p = joint_stats[1]

In [743]:
table4.loc['Joint F-stat'] = [np.nan] * (table4.shape[1]-1) + [f]

In [745]:
table4.loc['p-value (Prob>F)'] = [np.nan] * (table4.shape[1]-1) + [p]

In [746]:
table4

,All industry-years Mean,All industry-years STD,Run-ups Mean,Run-ups STD,Run-ups with Crash Mean,Run-ups with Crash STD,Run-ups with no Crash Mean,Run-ups with no Crash STD,Crash minus no Crash Mean,Crash minus no Crash [t]
Past 2-year Return,0.444960,0.509995,0.371353,0.389036,0.288414,0.309515,0.435861,0.438931,-0.147447,2.073163
Excess Past 2-year Return,0.413020,0.511321,0.324820,0.390064,0.238836,0.310051,0.391696,0.439446,-0.152861,8.028279
Volatility (VW),0.021285,0.071431,0.029554,0.014638,0.036379,0.018122,0.024245,0.008450,0.012134,3.653230
Volatility (VW)- 1yr-Δ,0.124993,3.051425,0.522543,1.410202,0.951792,2.038262,0.188683,0.423865,0.763109,-2.624837
Turnover (VW),0.061782,0.082721,0.123920,0.149272,0.186337,0.199062,0.075373,0.068016,0.110964,-0.333665
Turnover (VW)- 1yr-Δ,0.237634,2.307096,0.528438,0.867438,0.634109,1.038842,0.446249,0.728354,0.187859,-4.028050
Firm Age (VW),0.520568,0.319867,0.355696,0.291736,0.304634,0.226744,0.395410,0.334764,-0.090776,4.746137
Age tilt,-0.012371,0.093780,-0.113523,0.085789,-0.095928,0.099507,-0.127207,0.073454,0.031280,-1.528591
% Issuers,0.002972,0.012601,0.011646,0.028416,0.013830,0.035214,0.009948,0.022743,0.003882,-1.573267
Book to Market (VW),1.034296,5.590733,0.518099,0.864790,0.827575,1.222097,0.277396,0.285349,0.550178,-1.908779
